IMPORTS

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
# from keras.utils.vis_utils import plot_model
from matplotlib import pyplot as plt
np.set_printoptions(precision=3, linewidth=2*75, suppress=False)
from IPython.display import clear_output
from rise_and_fall import *
from dataset import Dataset
import json
import time
from neural_network import *
from main import *

PLOT FUNTIONS

In [ ]:
def plot_frame(f:pd.DataFrame, mxis=[], mnis=[], mark=None):
    ps = f["meanp"].to_numpy()
    mn = ps.min()
    mx = ps.max()
    plt.plot(ps)
    if len(mxis)>0:
        plt.vlines(x = mxis, ymin = mn, ymax = mx, colors = 'green',)
    elif 'mxis' in f.columns:
        plt.vlines(x = f.index[f['mxis']>0], ymin = mn, ymax = mx, colors = 'green',)
    if len(mnis)>0:
        plt.vlines(x = mnis, ymin = mn, ymax = mx, colors = 'red',)
    elif 'mnis' in f.columns:
        plt.vlines(x = f.index[f['mnis']>0], ymin = mn, ymax = mx, colors = 'red',)
    if mark is not None:
        plt.vlines(x = [mark], ymin = mn, ymax = mx, colors = 'blue',)
    plt.show()
    
def unique(xs):
    ys = []
    for x in xs:
        if x in ys:
            continue
        ys.append(x)
    return ys

SAVING DATASETS IN NPY

In [ ]:
# db = Dataset()
# if __name__=='__main__':
#     inputs, outputs = db.get_data_set(
#             batch_size=2**14, max_procs=10, single_process_size=2**10, buy=True, validation=False, test=False)

#     vlx, vly = db.get_data_set(
#             batch_size=2**11, max_procs=10, single_process_size=2**10, buy=True, validation=True, test=False)

# #     lrx, lry = db.get_data_set(
# #             batch_size=2**10, max_procs=10, single_process_size=2**9, buy=True, validation=False, test=False)

# np.save('database/inputs.npy', inputs)
# np.save('database/outputs.npy', outputs)

# np.save('database/vlx.npy', vlx)
# np.save('database/vly.npy', vly)

# np.save('database/lrx.npy', lrx)
# np.save('database/lry.npy', lry)

RETRIVING DATASET

In [ ]:
dsx_name = 'vlx'
dsy_name = 'vly'

ds = np.load(f'database/{dsx_name}.npy', mmap_mode='r')
indicies = np.random.choice(ds.shape[0], (2**10,))
x = ds[indicies]
del ds
print(x.shape)

ds = np.load(f'database/{dsy_name}.npy', mmap_mode='r')
y = ds.T[indicies]
y=y.T
del ds
print(y.shape)

PRETRAIN

In [8]:
inputs, outputs = get_data('inputs', 'outputs', 2**14)
vlx, vly = get_data('vlx', 'vly', 2**11)
lrx, lry = get_data('inputs', 'outputs', 2**10)

# lr = get_op_lr(lrx=lrx, lry=lry, mini_batch_size=32)
lr = 0.05
train_bp_model(inputs=inputs, outputs=outputs,
                       validation_data=(vlx, vly), epochs=5, verbose=1, lr=lr, mini_batch_size=512, patience=2)

# model = get_bp_model(lr)
# hist = model.fit(inputs, outputs, validation_data=(vlx,vly), epochs=5, batch_size=256)

Generated New Model
Train on 16384 samples, validate on 2048 samples
Epoch 1/5
16384/16384 [==============================] - 240s 15ms/sample - loss: 1910.6971 - y1_loss: 952.8832 - y2_loss: 943.6972 - y3_loss: 14.1167 - val_loss: 3095.9038 - val_y1_loss: 1000.0000 - val_y2_loss: 1000.0000 - val_y3_loss: 1095.9038
Epoch 2/5
12800/16384 [======================>.......] - ETA: 47s - loss: 1944.8287 - y1_loss: 941.5505 - y2_loss: 999.3104 - y3_loss: 3.9677

KeyboardInterrupt: 

MODEL TESTING

In [ ]:
model = get_bp_model(3e-4)
yp = model.predict(x)
ya = [y[i] for i in range(y.shape[0])]
plt.figure(figsize=(15,4))
for i in range(y.shape[0]):
    plt.subplot(1,3,i+1)
    plt.scatter(ya[i], yp[i])

RANDOM

In [ ]:
model = get_bp_model(1e-3)
model.summary()

In [ ]:
x=tf.constant([10.0 for _ in range(10)])
x.get_shape().as_list()[0]

In [ ]:
tf.constant([i for i in range(10)])

In [ ]:
with open("datasplit.json", 'r') as file:
        dataset = json.load(file)

In [ ]:
db = Dataset()

In [ ]:
a,b,c,d = db.labeled_buy_frame(at=100000-288, length=288, to_numpy=True)
display(a.shape)
# plot_frame(a)

In [ ]:
aa = []
size = 0
look_past = 0
while size<32:
    a,b,c,d = db.labeled_buy_frame(at=100000-look_past, length=288, to_numpy=False)
    ai = a.loc[np.logical_or(a['mnis']==1, a['mxis']==1), :]
    temp_a = np.zeros((ai.shape[0], 7))
    for i in range(1, ai.index.shape[0]):
        temp_a[i, 0:4] = a.loc[ai.index[i-1]:ai.index[i-1], ['meanp', 'ch', 'taker', 'maker']].mean().to_numpy().flatten()
        temp_a[i, 4:6] = a.loc[ai.index[i-1]:ai.index[i-1], ['vol', 'ntrds']].sum().to_numpy().flatten()
        temp_a[i, 6] = a.loc[ai.index[i-1]:ai.index[i-1], 'meanp'].std()
    
    temp_a =pd.DataFrame(temp_a, columns=['meanpp', 'chp', 'takerp', 'makerp','vols', 'ntrdss', 'meanpsstd'])
    ai.reset_index(drop=True, inplace=True)
    pd.concat([ai, temp_a], axis=1)

    aa.append(a)
    size += a.shape[0]
    look_past+=288
aa.reverse()
a = pd.concat(aa)
a=a.iloc[-32:]
a.reset_index(drop=True, inplace=True)
day = a.pop('day')
a['stime'] = a['stime']+day*18.0


In [ ]:
a

In [ ]:
a,b,c,d = db.labeled_buy_frame(at=100000-look_past, length=288, to_numpy=False)

In [ ]:
mu = a.to_numpy().mean()
sigma = a.to_numpy().std()
(a-mu)/sigma

In [ ]:
ai.index

In [ ]:
a.loc[6:10, 'meanp'].std()

In [ ]:
a

In [ ]:
plot_frame(a)

In [ ]:
# initial_learning_rate = 0.005
# decay_rate=0.87
# decay_steps=1024*5
# y = lambda step: initial_learning_rate * (decay_rate ** (np.round(step / decay_steps, 0)))

# steps = np.array([i for i in range(int(100*1024))])
# plt.semilogy(steps, y(steps))
# print(y(steps)[-1])

In [ ]:
print(get_bp_model(1).summary())
lrs = lambda epoch: 1e-4 * 10**(epoch/20)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lrs)
history = train_bp_model(inputs=x, outputs=[y[i] for i in range(y.shape[0])], epochs=60, verbose=0, save=False, patience=20, mini_batch_size=128)
losses = history.history['loss']
epochs = np.array([i for i in range(len(losses))])
plt.semilogx(lrs(epochs), history.history['loss'])

In [ ]:
lrs(epochs)[np.argmin(losses)]

In [ ]:
lambda step: initial_learning_rate * (decay_rate ** (np.round(step / decay_steps, 0)))

In [ ]:
plt.plot(np.array([i*np.log(i+1) for i in range(100)]))

In [ ]:
y=0.1
ys = []
for i in range(100):
    ys.append(y)
    # y = y*0.97**(i**2.6/(i+1)**2.4)
    y = 0.05*0.9**(12*np.log(i+1))
print(ys[-1])
plt.plot(ys)

In [ ]:
plt.plot([i**2.0 for i in range(10)])

In [ ]:
plt.plot([64*np.log(i+1) for i in range(63)])

In [ ]:
plt.plot([2*np.exp(i**(0.95**i)) for i in range(10)])

In [ ]:
[i*18.0 for i in range(7)]

In [ ]:
# model = buy_predictor()
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
# model.summary()